# TODO
- Parallelize cluster_approach over states
- Parallelize grid_approach over states
- Make two different .py files: cluster approach, grid approach.
- Make into folder: generating 'bounding_boxes'

- Make a separate prep_herb.py

- Correct eps size into haversine distance calculations

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

In [2]:
# Get Herbarium data. Find some candidate locations.
occ = pd.read_csv("/data/cher/universe7/herbarium/data/MO-herbarium/occurrence.txt", sep="\t", on_bad_lines='skip')
occ = occ[[
    'gbifID', 'occurrenceID', # Joining Keys
    'speciesKey', 'species', # species
    'year', 'month', 'day', # date
    'habitat', 'locality', # descriptive text. I think this is most of the route / habitat info we would want. Looks like already OCR processed?
    'countryCode', 'stateProvince', 'county', 'municipality', # administrative
    'georeferenceSources','hasCoordinate', 'hasGeospatialIssues' ,'decimalLatitude', 'decimalLongitude']] # geospatial
    # 'level0Gid', 'level0Name', 'level1Gid', 'level1Name', 'level2Gid', 'level2Name', 'level3Gid', 'level3Name',]] # Has NAs. Above is more reliable.

habitat_info = occ[~occ['habitat'].isna()].copy()

/tmp/ipykernel_329770/903731494.py:2: DtypeWarning: Columns (49,56,57,59,60,61,63,64,65,66,67,68,73,74,77,78,79,83,89,91,92,95,97,98,99,100,101,102,103,105,106,107,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,135,137,138,140,141,142,143,144,145,146,147,149,150,151,152,153,154,155,160,162,163,164,167,168,171,173,174,177,178,182,183,184,185,186,189,190,191,192,193,194,195,196,197,198,199,200,204,208,209,210,211,220,221) have mixed types. Specify dtype option on import or set low_memory=False.
  occ = pd.read_csv("/data/cher/universe7/herbarium/data/MO-herbarium/occurrence.txt", sep="\t", on_bad_lines='skip')


In [4]:
# Random location
random_row = habitat_info[~habitat_info['habitat'].isna()].iloc[-4]
lat, lon = random_row['decimalLatitude'], random_row['decimalLongitude']

NAIP 
- Only 3 band RGB through WMS

In [13]:
import urllib.request
import rasterio
import matplotlib.pyplot as plt
import numpy as np


image_url = f"https://basemap.nationalmap.gov/arcgis/services/USGSImageryOnly/MapServer/WMSServer?service=WMS&version=1.1.1&request=GetMap&layers=0&styles=&width=256&height=256&srs=EPSG:4326&bbox={lon-0.005},{lat-0.005},{lon+0.005},{lat+0.005}&format=image/png"
out_file = f'/data/cher/universe7/herbarium/data/naip/1.png'

urllib.request.urlretrieve(image_url, out_file)

with rasterio.open(out_file) as src:
    # Get the number of bands
    num_bands = src.count
    print(f'The TIFF file has {num_bands} bands.')

('/data/cher/universe7/herbarium/data/naip/1.tiff',
 <http.client.HTTPMessage at 0x7f105f5980e0>)

Sentinel2
- Create csv file to push into CVGlobal 
    1) Cluster points state-by-state using DBSCAN into max areas of 512 x 512 meters --> cell
    2) Find center point of each cell

#### USA

In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, box
from sklearn.cluster import DBSCAN
import numpy as np
from tqdm import tqdm
import os

1) DBSCAN still generates clustered grid cells outside of 512 meters.
- Instead we will do the following --> 
    1) generate DBSCAN clusters. Find points within those centers. 
    2) Points outside centers --> DBSCAN. Find points within those centers.

2) Fast --> Create grid for state. Find overlap of points. Return just those grids

In [134]:
# Get general info.
num_bbox_per_state = clusters['state_cluster'].str.split('_').str[0].value_counts()
num_images_per_state = cluster_assignments['state_cluster'].str.split('_').str[0].value_counts()
num_images_per_cluster = num_images_per_state / num_bbox_per_state

print(num_bbox_per_state, num_images_per_state, num_images_per_cluster)

display(num_images_per_cluster.sort_values(ascending = False))

state_cluster
Michigan    788
Name: count, dtype: int64 state_cluster
Michigan    4479
Name: count, dtype: int64 state_cluster
Michigan    5.68401
Name: count, dtype: float64


state_cluster
Michigan    5.68401
Name: count, dtype: float64

In [1]:
import pandas as pd
import geopandas as gpd
from scipy.spatial.distance import pdist, squareform
from shapely.geometry import Point

In [31]:
grid = gpd.read_file("/data/cher/universe7/herbarium/data/geocell/grid.geojson")
grid_key = pd.read_csv("/data/cher/universe7/herbarium/data/geocell/grid_key.csv")

In [71]:
### sanity checks
# Are places assigned to the same cluster within 512 meters of one another?
habitat_gdf_w_bbox_cluster = gdf.merge(cluster_assignments, on = ['occurrenceID'])

ex = habitat_gdf_w_bbox_cluster[habitat_gdf_w_bbox_cluster['stateProvince'] == 'New York']

In [ ]:
# Use grid cells across US that have occurrences for query of images.
# sentinal2_key.csv -- Source - gbif, key - numeric_key, lat, lon  
# obs_id2key .csv -- [Source, obs_id, key, bbox_bounds]

In [17]:
# TODO: Parallelize this and loop across states --> grid.py
#  ## Inefficient method -- Grid across state. point-in-polygon.
    # 1) Create equal spatial grid of CA (512M x 512M)
    # 2) Find center point of each cell
    # 3) Remove grids outside CA
    # 4) Calculate # of cells with observation
    # 5) Calculate # of observations per cell

# # California ~4 min

# ## Inefficient method
# import geopandas as gpd
# import pandas as pd
# import numpy as np
# from shapely.geometry import box, Point
# from joblib import Parallel, delayed
# from tqdm import tqdm

# def build_grid(cell_size_m, bbox):
#     # Approximate conversion factors
#     deg_per_meter_lat = 1 / 111000  # degrees per meter for latitude
#     deg_per_meter_lon = 1 / (111000 * np.cos(np.radians((bbox[1] + bbox[3]) / 2)))  # degrees per meter for longitude

#     # Calculate the number of rows and columns
#     lat_range = bbox[3] - bbox[1]  # degrees of latitude
#     lon_range = bbox[2] - bbox[0]  # degrees of longitude

#     nrows = int(np.ceil(lat_range / (cell_size_m * deg_per_meter_lat)))
#     ncols = int(np.ceil(lon_range / (cell_size_m * deg_per_meter_lon)))

#     # Use tqdm to create a progress bar
#     tasks = [(i, j, cell_size_m, deg_per_meter_lat, deg_per_meter_lon, bbox) for i in range(nrows) for j in range(ncols)]
#     with Parallel(n_jobs=-1) as parallel:
#         grid_polygons = list(tqdm(parallel(delayed(create_cell)(*task) for task in tasks), total=len(tasks)))

#     return grid_polygons

# def create_cell(i, j, cell_size_m, deg_per_meter_lat, deg_per_meter_lon, bbox):
#     # Calculate the corners of the cell
#     min_lat = bbox[1] + i * cell_size_m * deg_per_meter_lat
#     max_lat = bbox[1] + (i + 1) * cell_size_m * deg_per_meter_lat
#     min_lon = bbox[0] + j * cell_size_m * deg_per_meter_lon
#     max_lon = bbox[0] + (j + 1) * cell_size_m * deg_per_meter_lon

#     # Create a polygon for the cell
#     return box(min_lon, min_lat, max_lon, max_lat)

# california = gpd.read_file('/data/cher/universe7/herbarium/data/California_State_Boundary.geojson')
# grid_polygons = build_grid(cell_size_m = 1024, bbox = california.total_bounds)

# # Create a GeoDataFrame for the grid
# grid_gdf = gpd.GeoDataFrame(geometry=grid_polygons)
# grid_gdf.set_crs(california.crs, inplace=True)
# grid_within_ca = gpd.sjoin(grid_gdf, california, how='inner', predicate='intersects')

# # Create a DataFrame with center points of remaining cells
# grid_within_ca = gpd.GeoDataFrame(grid_within_ca[['geometry']], geometry='geometry')
# grid_centers = grid_within_ca.geometry.apply(lambda geom: geom.centroid)
# centroid_points = pd.DataFrame(grid_centers.apply(lambda point: (point.x, point.y)).tolist(), columns=['Longitude', 'Latitude'])

# CA_data = habitat_info[habitat_info['stateProvince'] == 'California']
# # Remove rows with non-numeric latitudes or longitudes
# CA_data = clean_data(CA_data[['occurrenceID','stateProvince','decimalLatitude', 'decimalLongitude']].copy())
# CA_data_gdf = gpd.GeoDataFrame(CA_data, geometry='geometry')

# CA_data_gdf.set_crs(grid_within_ca.crs, inplace=True)
# observations_in_cells = gpd.sjoin(CA_data_gdf, grid_within_ca, how='inner', predicate='within')

# # Calculate the number of cells with at least one observation
# num_cells_with_observations = observations_in_cells['index_right'].nunique()
# observations_per_cell = observations_in_cells.groupby('index_right').size().reset_index(name='observations_count').sort_values('observations_count', ascending = False)

# print(f"Occurrences: {CA_data.shape[0]}\nSatellite images: {num_cells_with_observations}\nDescriptions per image {np.mean(observations_per_cell.observations_count)}")

Occurrences: 12654
Satellite images: 4798
Descriptions per image 2.634014172571905
